In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
import numpy as np

import matplotlib.pyplot as plt
from torchvision import datasets, transforms

In [15]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=0.5, std=0.5),
                              ])

trainset = datasets.MNIST('MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

for images, labels in trainloader:
    print(type(images))
    print(images.shape)
    print(labels.shape)
    break
    
inputs = images.view(images.shape[0], -1)
print(inputs.shape)

<class 'torch.Tensor'>
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 784])


In [27]:
n_input = inputs.shape[1]
n_hidden = 256
n_output = len(torch.unique(labels))
print(n_input)

784


In [46]:
def softmax (scores):    
    return torch.div(torch.exp(scores), torch.sum(torch.exp(scores), dim=1).view(-1,1))

In [47]:
w1 = torch.randn(n_input, n_hidden)
b1 = torch.randn(n_hidden)

w2 = torch.randn(n_hidden, n_output)
b2 = torch.randn(n_output)

h  = activation(torch.mm(inputs, w1) + b1)
output = softmax(torch.mm(h, w2) + b2)

In [48]:
print(output.shape)
print(output.sum(dim=1))

torch.Size([64, 10])
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])
